In [34]:
import pandas as pd

In [35]:
df = pd.read_csv("dpschool2a.csv", delimiter=";")

In [45]:
df.drop(df[df["latitude"] == "0"].index, inplace)

Int64Index([843, 848, 858, 961, 1016, 1033, 1086, 1087, 1088, 1089, 1090,
            1091],
           dtype='int64')

Int64Index([843, 848, 858, 961, 1016, 1033, 1086, 1087, 1088, 1089, 1090,
            1091],
           dtype='int64')

1033    0
Name: latitude, dtype: object

In [40]:
df.head()

,title,short_description,description,latitude,longitude,address,addr_no,zip_code,city,country,additional_fields,image,icons,modified
1,WC Bad Georgenschwaige Damen,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Damen-WC</strong> ist im Umklei...,4.817.566,1.157.174,Belgradstr.,195,80804,MÃ¼nchen,DE,"{""wc_shortname"":""ba-george1WC1"",""wc_plan"":""ba-...","{""image_small"":""images\/wc-af\/ba-george1WC1f1...","{""0"":""4"",""1"":""11""}",09/08/2021 09:21
2,WC Bad Georgenschwaige Herren,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Herren-WC ist im Umkleidebau</s...,4.817.566,1.157.174,Belgradstr.,195,80804,MÃ¼nchen,DE,"{""wc_shortname"":""ba-george1WC2"",""wc_plan"":""ba-...","{""image_small"":""images\/wc-af\/ba-george1WC2f1...","{""0"":""6"",""1"":""20""}",09/08/2021 09:21
3,WC Freibad Unterhaching,Bedingt Ã¶ffentliche Toilette mit Dusche in ei...,<p>Das rollstuhlgerechte WC des Unterhachinger...,480.619.761,116.208.199,Schrenkstr.,2,82008,Unterhaching,Deutschland,"{""wc_shortname"":""ba-unterh1WC"",""wc_plan"":""ba-u...","{""image_small"":""images\/wc-af\/ba-unterh1WC1f1...","{""0"":""4"",""1"":""8"",""2"":""11""}",10/06/2021 08:10
4,WC U-Bahnhof MÃ¼nchner Freiheit,"Ã–ffentliche Toilette im U-Bahnhof, zugÃ¤nglic...",<p>Das rollstuhlgerechte WC des U-Bahnhofs MÃ¼...,481.622.924,115.870.025,MÃ¼nchner Freiheit,NaN,80802,MÃ¼nchen,DE,"{""wc_shortname"":""bf-muefre1WC"",""wc_plan"":""bf-m...","{""image_small"":""images\/wc-af\/bf-muefre1WC1f1...","{""0"":""4"",""1"":""11"",""2"":""12""}",10/06/2021 08:09
5,WC Dynasty Unterhaching,HalbÃ¶ffentliche Toilette in einem Lokal.,<p>Auf dem Weg zum Haupteingang ist eine Schwe...,480.556.938,116.076.273,Inselkammerstr.,3,82008,Unterhaching,Deutschland,"{""wc_shortname"":""dynast1"",""wc_plan"":""dynast1WC...","{""image_small"":""images\/wc-af\/dynast1WC1f1.jp...","{""0"":""4"",""1"":""17""}",18/08/2021 16:24


In [3]:
df.drop(df[(df["latitude"] == 0) | df["longitude"] == 0].index, inplace=True)

In [37]:
df = df[df["published"] == 1]

In [38]:
df = df[['title', 'short_description', 'description', 'latitude', 'longitude', 'address', 'addr_no', 
         'zip_code', 'city', 'country', 'additional_fields', 'image', 'icons', 'modified']]

In [6]:
import json
df["additional_fields"] = df["additional_fields"].map(lambda x: json.loads(x))
df["image"] = df["image"].map(lambda x: json.loads(x))

In [7]:
df = df[df['latitude'].notna()]

In [8]:
from pandas import isnull
df["icons"] = df["icons"].map(lambda x: {} if isnull(x) else x)
df["icons"] = df["icons"].map(lambda x: x if isinstance(x, str) else str(x))
df["icons"] = df["icons"].map(lambda x: json.loads(x))

In [9]:
df["modified"] = pd.to_datetime(df["modified"], errors="coerce")

In [10]:
df['longitude'] = df['longitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000
df['latitude'] = df['latitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000

In [11]:
df["description"] = df["description"].map(lambda x: "" if isnull(x) else x)
df["short_description"] = df["short_description"].map(lambda x: "" if isnull(x) else x)
df["addr_no"] = df["addr_no"].map(lambda x: "" if isnull(x) else x)

In [12]:
df["address"] = df["address"] + " " + df["addr_no"]

In [13]:
df.drop(["addr_no"], axis=1, inplace=True)

In [14]:
df["position"] = None
for i in df.index:
    df.at[i, 'position'] = (df.loc[i]['latitude'], df.loc[i]['longitude'])

In [15]:
icons = {"Toilette für Alle": 10, "Rolli WC": 11, "Rolli WC -DIN fern": 20, "WC eng": 17, "kein Rolli WC vorhanden": 21,
        "Parken": 7, "Stufe": 1, "WC - unisex": 4, "WC getrennt": 6, "Rollstuhl": 5, "Lift groß": 12, "Lift klein": 13,
        "Kino-Untertitel": 15, "Induktionsschleife": 14, "low-vision": 18, "kognitiv beeinträchtigt": 19, "Umkleide": 9,
        "Dusche": 8, "nicht rollstuhlgerecht": 16, "Rampe": 2, "keine Steigung - eben": 26, "moderate Steigung / Gefälle": 22,
        "weitere moderate Steigungen / Gefälle": 23, "heftige Steigung / Gefälle": 25, "weitere heftige Steigung / Gefälle": 24}
new_dict = {v: k for k, v in icons.items()}
new_dict.update({40: "Euro Key", 41: "Ramp Incline", 42: "Door Width"})

In [16]:
df["icons"] = df["icons"].map(lambda x: tuple(map(int, list(x.values()))))

In [17]:
df[40] = df["additional_fields"].map(lambda x: x["wc_euro"] if "wc_euro" in x.keys() else "0")
df[40] = df[40].map(lambda x: int(x) if x in ["1", "0"] else 0)

In [18]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

df[41] = df["additional_fields"].map(lambda x: x["wc_ramp_incline"])
df[41] = df[41].map(lambda x: x.replace(",","."))
df[41] = df[41].map(lambda x: float(x) if isfloat(x) else 0)
#need to check further as some have extra text

In [19]:
df[42] = df["additional_fields"].map(lambda x: x["wc_door_width"])
df[42] = df[42].map(lambda x: int(x) if x != "" else 150)
df[42] = df[42].map(lambda x: x if x > 15 else x*10)

In [20]:
x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)
x = x.rename(int, axis="columns")
col = list(x.columns)
for i in [40, 41, 42]:
    col.append(i)

In [21]:
df = df.rename(columns={40:"eurokey", 41: "ramp_steepness", 42: "door_width"})

In [22]:
df["photo"] = df["image"].map(lambda x: x["image_small"])
df["plan"] = df["additional_fields"].map(lambda x: "images/wc-plan/" + x["wc_plan"] + ".jpg")
#df["seat_height"] = df["additional_fields"].map(lambda x: x["wc_seat"])

In [23]:
df = df.drop(df.loc[df['icons'].isin([(21,)])].index)

In [24]:
df["grip"] = df["additional_fields"].map(lambda x:x["wc_wheregrip"])

lr = flexible left & flexible right

l = flexible left

rfl & flr = fixed left and flexible right

fr = fixed right

none = no grip

lfr = flexible left and fixed right

r = flexible right

fl = fixed left

In [25]:
df["access"] = df["additional_fields"].map(lambda x:x["wc_infos"])
df["interior_description"] = df["additional_fields"].map(lambda x:x["wc_roominfos"])
df["wc_details"] = df["additional_fields"].map(lambda x:x["wc_addinfo"])
df["wc_accessleft"] = df["additional_fields"].map(lambda x:x["wc_accessleft"])
df["wc_accessright"] = df["additional_fields"].map(lambda x:x["wc_accessright"])

In [27]:
df[df["title"] == "WC Bahnhof Buchenau"]

,title,short_description,description,latitude,longitude,address,zip_code,city,country,additional_fields,...,ramp_steepness,door_width,photo,plan,grip,access,interior_description,wc_details,wc_accessleft,wc_accessright
1033,WC Bahnhof Buchenau,Ã–ffentliche Bahnhofstoilette mit Euroschloss.,,0.0,0.0,Kurt-Huber-Ring 13a,82265,FÃ¼rstenfeldbruck,DE,"{'wc_shortname': 'bf-buchen1WC1', 'wc_plan': '...",...,0.0,90,images/wc-af/bf-buchen1WC1f2.jpg,images/wc-plan/bf-buchen1WC1.jpg,lr,Die Tür zur rollstuhlgerechten bzw. behinderte...,"Das Waschbecken, gegenüber von der Tür, ist au...",Die Spültaste ist hinter dem WC-Becken in 100 ...,110,80


In [ ]:
df = df.rename(columns={"description": "direction"})

In [ ]:
df = df[["title", "short_description", "photo", "plan", "position", "address", "eurokey", "ramp_steepness", "door_width", "modified",
         "zip_code", "city", "grip", "direction", "access", "interior_description", "wc_details", "wc_accessright", "wc_accessleft"]]

In [ ]:
errors = pd.read_excel("toilets_text errors.xlsx")

In [ ]:
errors.dropna(inplace=True)

In [ ]:
list(errors["Text error"])

In [ ]:
import re

errors = pd.Series(errors["correction"].values,index=errors["Text error"]).to_dict()

# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in errors.items()) 
#Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))

In [ ]:
df["short_description"] = df["short_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["title"] = df["title"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["address"] = df["address"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["city"] = df["city"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["direction"] = df["direction"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["access"] = df["access"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["interior_description"] = df["interior_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["wc_details"] = df["wc_details"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
df["direction"] = df["direction"].map(lambda x:remove_html_tags(x))
df["access"] = df["access"].map(lambda x:remove_html_tags(x))
df["interior_description"] = df["interior_description"].map(lambda x:remove_html_tags(x))
df["wc_details"] = df["wc_details"].map(lambda x:remove_html_tags(x))

In [ ]:
df["id"] = df.index

In [ ]:
df.head()

In [ ]:
df.to_json("toilets_v14.json")